In [10]:
import requests, csv #Set variables year='2019' dsource='pep'
import us
import pandas as pd


In [4]:
host = 'https://api.census.gov/data'

Constructing County Calls

In [7]:
year = '2019'
dataset = 'acs/acs1'
g = '?get='
variables = 'NAME,B01001_001E'
location = '&for=state=17&county=*'
series = "B01001_001E"
state = 17

counties = [31, #Cook
    43, #Dupage
    89, #Kane
    97, #Lake
    111, #McHenry
    197 #Will
]




In [12]:
pd.read_json("https://api.census.gov/data/2021/acs/acs1?get=NAME,B01001_001E&for=county:*&in=state:17",)

,0,1,2,3
0,NAME,B01001_001E,state,county
1,"Kankakee County, Illinois",106601,17,091
2,"Lake County, Illinois",711239,17,097
3,"McLean County, Illinois",170889,17,113
4,"Madison County, Illinois",264490,17,119
5,"St. Clair County, Illinois",254796,17,163
6,"Tazewell County, Illinois",130413,17,179
7,"Winnebago County, Illinois",283119,17,201
8,"Champaign County, Illinois",205943,17,019
9,"Cook County, Illinois",5173146,17,031


In [ ]:
responses = []
for year in range(2005, 2022):
    data_url = f'https://api.census.gov/data/{year}/{dataset}?get=NAME,{series}&for=county:*&in=state:{state}'
    df = pd.read_json(data_url)
    df.Year = year
    responses.append

In [8]:
data_url

'https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001_001E&for=county:*&in=state:17'

In [ ]:
https://api.census.gov/data/2021/acs/acs1?get=NAME,B01001_001E&for=county:*&in=state:17
https://api.census.gov/data/2019/acs/acs1/?get=NAME,B01001_001E&for=county:*&in=state=17 -> Me

In [ ]:
https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001_001E&for=county:*&key=YO

In [ ]:


dname='population' cols= 'NAME, POP, DATE_DESC state='44' county='*
dcode= '2, 12' keyfile='census_key.txt'
outfile='pop2019 ri_counties.txt'
base_ur = f'https://api.census.gov/data/fyear}/{dsource}/{dname}' #Read api key in from file
with open (keyfile) as key:
api key=key. read() .strip ()
#Retrieve data, print output to screen
data_url = f'{base_url}?get={cols}&DATE_CODE={dcode}&for=county:{county}\ Jin-state:(stater&key-(api keys"
response=requests.get (data_url) popdata=response.json ()
for record in popdata:
print (record) #Write data to CSV
with open (outfile,
'w', newline='') as writefile:
writer = csv.writer(writefile, quoting=csv. QUOTE_ALL, delimiter=',')
writer.writerows (popdata)